In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
ALL_EVALUATIONS="tests/evaluation_10_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240108.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"
E5_RUN="tests/e5_docs_000_009_50_sampled_queries_score_inv_run.txt"
E5_BASE_RUN="tests/e5_base_docs_000_009_50_sampled_queries_score_inv_run.txt"
BM25_1M_RUN="tests/bm25_1M_docs_000_009_sampled_50_queries_run.txt"
BM25_1M_MASSIVEWEB_RUN="tests/bm25_massiveweb_1M_docs_000_009_sampled_50_queries_run.txt"

In [4]:
SPLADE_MT5="tests/splade_mt5_docs_000_009_50_sampled_queries.tsv"

In [5]:
retrievers_results = []

### Format GPT-4 evaluations to qrels format

In [6]:
eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

In [7]:
eval_df

,query_id,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,e5,e5_base,bm25_1M,bm25_1M_massiveweb,splade_mT5,passage,score,reasoning,usage,cost,duration,saved_cost
0,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-82-13187_0,False,False,True,False,False,False,False,False,False,False,"Mamíferos do Brasil - Zoologia | Museu Nacional - UFRJ ➤ Início ➤ Exposições ➤ Zoologia (Acervo) ➤ Insetos ➤ Mamíferos do Brasil Mamíferos do Brasil Uma larga coleção de mamíferos da fauna brasileira. Entre os importantes representantes deste acervo de espécimes taxidermizados encontram-se grandes símbolos da diversidade biológica do Brasil. Bugio, Barbado, Guariba Bugio Cachorro-do-mato Caititu Capivara Caxinguelê, Serelepe Cervo-do-Pantanal Gato-do-Mato-pequeno Irara Lobo-Guará Lontra Mão-Pelada Muriqui Onça-Pintada Ouriço-Cacheiro, Cuandu Paca Preguiça-de-Bentinho Preguiça-de-três-dedos Rato-Coró Saruê, Gambá Sauá, Guigó Tamanduá-bandeira Tamanduá-í Tamanduá-mirim, meleta Tatu-Canastra Tatu-de-rabo-mole Tatu-Galinha Tatu-Peludo Veado Veado-mateiro « Voltar | Voltar ao Topo",1,"A passagem lista vários mamíferos da fauna brasileira, mas não descreve características gerais da fauna brasileira, como a pergunta solicita. A passagem está relacionada ao tema da fauna brasileira, mas não responde diretamente à pergunta sobre a maior característica da fauna.","{'prompt_tokens': 765, 'completion_tokens': 75, 'total_tokens': 840}",0.00000,6.693229,0.02745
1,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-59-04599_2,False,False,True,False,False,False,False,False,False,False,"Segundo uma lista levantada pelo Comitê Brasileiro de Registros Ornitológicos (CBRO), algumas das principais espécies encontradas aqui no país são: Bem-te-vi, Arara-azul, Tuiuiú, Tucano e o Papagaio. Só no Cerrado pode-se encontrar por volta de 148 espécies de aves bem específicas da região. Algumas das aves que mais representam esta região brasileira são: Seriemas; Pavãozinho-do-pará; Coruja-orelhuda; Arara-canindé; Coruja-da-igreja; Arara-vermelha-grande; Tuiuiú (considerado o símbolo do Pantanal). Quando chegamos à região nordestina a ave brasileira já foi até tema de música, a famosa ave com seu voo incrível chamada “Asa branca”, ave que deu nome à música de mais sucesso na voz do cantor e compositor Luiz Gonzaga. A Amazônia é uma região brasileira que qualquer um se encanta pelas belezas naturais principalmente quando o assunto são espécies de aves. As araras com sua pelagem colorida chamam a atenção de todos e podem ser vistas por todos os lados na região. A segunda ave que pode ser encontrada facilmente na Amazônia é o tucano, com suas características marcantes, um bico longo e cores vibrantes que marcam a região dos olhos. Claro que podemos encontrar essas bela espécie de ave em zoológicos espalhados por todo o país ou em áreas de matas em geral.",1,"A passagem descreve várias espécies de aves encontradas no Brasil e suas características, mas não resume qual é a maior característica da fauna brasileira de forma geral. A pergunta pede uma característica ampla da fauna, enquanto a passagem foca em exemplos específicos de aves em diferentes regiões.","{'prompt_tokens': 860, 'completion_tokens': 82, 'total_tokens': 942}",0.00000,5.163547,0.03072
2,1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-64-06285_39,True,True,False,False,False,False,True,False,False,False,"A exuberante fauna e flora brasileiras, relatadas por Pero Vaz de Caminha em sua carta à Coroa Portuguesa, está sendo destruída gradativamente pela ação antrópica, por meio do desmatamento e da caça d... Leia redação completa Redação sem título. 02/07/2016 O Brasil tem a maior biodiversidade do mundo. Devido a sua variedade de espécies silvestres, desencadeou uma enorme ambição global ao tráfico de animais desde o período colonial. Atualmente, tal ato a... Leia redação completa Redação sem título. 19/09/2018 O Brasil com seu vasto território é um dos países com a 

In [8]:
eval_df.drop_duplicates("query_id")[['query_id', 'query']]

,query_id,query
0,1,Qual a maior característica da fauna brasileira?
64,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
135,9,quais países europeus seguem o regime monarquista?
200,11,quais os critérios de definição dos monumentos intitulados maravilhas do mundo moderno?
255,13,Como o Brasil reagiu a epidemia de AIDS no fim do século XX?
309,15,Qual a maior torcida de futebol do Brasil?
353,17,Quando se realizou o plebiscito popular para definir o sistema político do Brasil?
404,20,Por que a legislação de um país é tão importante?
481,21,Qual o nosso atual sistema de governo?
554,22,Qual o sistema de governo da África do Sul?


#### Check scores distribution on the current qrels

In [9]:
np.unique(eval_df['score'].to_numpy(), return_counts=True)

(array([0, 1, 2, 3]), array([1465,  613,  474,  515]))

In [10]:
for group_name, group_df in eval_df.groupby('query_id'):
    print("\nquery_id={}".format(group_name))
    
    print(np.unique(group_df['score'].to_numpy(), return_counts=True))


query_id=1
(array([0, 1, 2, 3]), array([ 9, 35, 12,  8]))

query_id=2
(array([0, 1, 2, 3]), array([66,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([45,  6,  9,  5]))

query_id=11
(array([0, 1, 2, 3]), array([33, 12,  8,  2]))

query_id=13
(array([0, 1, 2, 3]), array([25, 10, 15,  4]))

query_id=15
(array([0, 1, 2, 3]), array([16, 14,  1, 13]))

query_id=17
(array([0, 1, 2, 3]), array([15,  7,  1, 28]))

query_id=20
(array([0, 1, 2, 3]), array([13, 39, 19,  6]))

query_id=21
(array([0, 1, 2, 3]), array([38, 18,  2, 15]))

query_id=22
(array([0, 1, 2, 3]), array([42, 10,  1,  6]))

query_id=24
(array([0, 1, 2, 3]), array([45, 17, 10,  2]))

query_id=26
(array([0, 1, 2, 3]), array([13,  8,  9, 27]))

query_id=28
(array([0, 1, 2, 3]), array([42, 11, 13,  4]))

query_id=47
(array([0, 1, 2]), array([35, 24, 19]))

query_id=49
(array([0, 1, 2, 3]), array([20, 20, 18,  1]))

query_id=51
(array([0, 1, 2, 3]), array([21, 17, 10,  1]))

query_id=54
(array([0, 1, 2, 3]), array([ 1, 12, 

In [11]:
eval_df.groupby('query_id').count()

,query,passage_id,mT5,splade,colbertx,bm25_e5,bm25,e5,e5_base,bm25_1M,bm25_1M_massiveweb,splade_mT5,passage,score,reasoning,usage,cost,duration,saved_cost
query_id,,,,,,,,,,,,,,,,,,,
1,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64,64
2,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71
9,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65,65
11,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55,55
13,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54,54
15,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44,44
17,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51,51
20,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77,77
21,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73,73


In [12]:
eval_df.groupby('query_id').count()['query'].describe()

count    50.000000
mean     61.340000
std      10.831944
min      35.000000
25%      52.250000
50%      61.000000
75%      69.750000
max      84.000000
Name: query, dtype: float64

#### Prepare the qrels format for trec_eval tools

In [13]:
eval_df['0'] = '0'

In [14]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [15]:
qrels_filename

'tests/evaluation_10_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240108_qrel.txt'

In [16]:
eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")

### Compute BM25 nDCG@10

In [17]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [18]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [19]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4546'}]

### Compute BM25 + E5 nDCG@10

In [20]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [21]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [22]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [23]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [24]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [25]:
retrievers_results.append({'retriever':'SPLADE_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [26]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [27]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [28]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [29]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 base nDCG@10

In [30]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {E5_BASE_RUN}

In [31]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M nDCG@10

In [32]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_RUN}

retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M massiveweb filtered nDCG@10

In [33]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare SPLADE + mT5 retrieval results in run format

In [34]:
splade_mt5_df = pd.read_csv(SPLADE_MT5, sep="\t")

In [35]:
splade_mt5_df.groupby("query_id").head(10).count()

query          500
passage_id     500
score          500
query_id       500
colbertx_id    500
passage        500
dtype: int64

In [36]:
splade_mt5_df

,query,passage_id,score,query_id,colbertx_id,passage
0,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-22-15255_0,1000,1,5877599,"Fauna do Brasil - Toda Matéria Biologia › Zoologia Fauna do Brasil A fauna do Brasil é provavelmente a que possui a maior diversidade em espécies de animais vertebrados, anfíbios e primatas do globo terrestre. Existem três vezes mais espécies de peixes de água doce no Brasil do que em qualquer lugar no mundo. As aves também são muito variadas, colocando o Brasil como o país com a maior variedade de espécies na América do Sul e o terceiro do mundo em espécies de mamíferos. De igual modo, a Floresta Atlântica está entre as cinco regiões do planeta com mais espécies nativas, das quais 17% são aves e 10% são anfíbios e mamíferos encontrados somente na fauna brasileira. Vale citar que os animais da fauna brasileira são, via de regra, de pequeno porte. A importância científica da fauna brasileira é indescritível, pois contém a maior biodiversidade do mundo (cerca de 100 mil espécies de invertebrados, 8200 espécies vertebrados (713 mamíferos, 1826 aves, 721 répteis, 875 anfíbios, 2800 peixes continentais e 1300 peixes marinhos) das quais 627 já estão na lista negra das espécies ameaçadas de extinção (394 espécies terrestres e 233 espécies aquáticas)."
1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-61-07564_63,990,1,2899447,"A fauna da Floresta Amazônica tem a maior riqueza e diversidade do Brasil.[71] Dentre os macacos são destacados os saís, os macacos-de-cheiro, os barrigudos, os guaribas e os saguis. São também bastante numerosos os cachorros-do-mato, quatis, furões, jaguatiricas e preguiças. O peixe-boi, o boto-branco e o tucuxi são animais de exclusividade dessa região. O jacamim, o pavãozinho-do-pará, a arara-vermelha, o pavão-do-mato são aves típicas da Amazônia, enquanto que dentre os ofídios é importante mencionar, por serem grandes, as jiboias e as sucuris. Dentre os peixes da Amazônia são salientados o pirarucu, as raias, tucunaré, poraquê e o baiacu. São encontrados também jacarés, tartarugas e o surubim (este em abundância ao sul da Amazônia).[71] A fauna pantaneira também se destaca quanto à diversidade de espécies animais, sendo o bioma considerado como um dos maiores centros de reprodução das Américas. Aves como o jaburu, a garça, a arara-azul-grande e o biguá; mamíferos como a onça-pintada, o cervo-do-pantanal e o lobo-guará; répteis como a sucuri; e peixes como a piramboia, o dourado e o lambari representam algumas das espécies típicas do ambiente pantaneiro.[143][144] O muriqui-do-norte é um primata endêmico do Brasil e considerado criticamente em perigo"
2,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-21-03309_40,980,1,1021792,"São surpreendentemente uniformes, pois suas características mais óbvias são as penas e o bico córneo. São chamadas de endotérmicas , porque produzem seu próprio calor e de homoeotérmicas , porque podem manter a temperatura de seus corpos razoavelmente alta e constante. Isto não significa que a temperatura do corpo de uma ave não varie, pode haver oscilação diária de vários graus. São tetrápodas, com o par anterior transformado em asas e o posterior adaptado para empoleirar, andar ou nadar. A capacidade de voar permite às aves ocupar alguns hábitats impossíveis para outros animais. Além da quantidade, a avifauna do Brasil reúne inúmeros superlativos quanto à qualidade. Vive aqui, uma das maiores aves do mundo, a ema, ao lado das aves de menor porte, os beija-flores. Encontram-se os voadores de maior porte da Terra: o albatroz e o condor, ambos de ocorrência apenas ocasional. O gavião-real, residente no Brasil, é a ave de rapina mais possante do mundo. Ocorrem aqui as aves de vôo mais veloz: falcões e andorinhões."
3,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-77-06884_1,970,1,3668516,"O tamanho do Brasil e a grande variedade de climas e características e suas regiões são o que torna p

In [37]:
splade_mt5_run_df = splade_mt5_df.sort_values(["query_id", "score"], ascending=[True, False]).groupby("query_id").head(10)

In [38]:
splade_mt5_run_df.shape

(500, 6)

In [39]:
splade_mt5_run_df['Q0'] = "Q0"
splade_mt5_run_df['comment'] = "gpt-4-1106-preview"
splade_mt5_run_df['index'] = np.tile(list(range(1, 11)), 50)

In [40]:
splade_mt5_run_df.head(12)

,query,passage_id,score,query_id,colbertx_id,passage,Q0,comment,index
0,Qual a maior característica da fauna brasileira?,clueweb22-pt0001-22-15255_0,1000,1,5877599,"Fauna do Brasil - Toda Matéria Biologia › Zoologia Fauna do Brasil A fauna do Brasil é provavelmente a que possui a maior diversidade em espécies de animais vertebrados, anfíbios e primatas do globo terrestre. Existem três vezes mais espécies de peixes de água doce no Brasil do que em qualquer lugar no mundo. As aves também são muito variadas, colocando o Brasil como o país com a maior variedade de espécies na América do Sul e o terceiro do mundo em espécies de mamíferos. De igual modo, a Floresta Atlântica está entre as cinco regiões do planeta com mais espécies nativas, das quais 17% são aves e 10% são anfíbios e mamíferos encontrados somente na fauna brasileira. Vale citar que os animais da fauna brasileira são, via de regra, de pequeno porte. A importância científica da fauna brasileira é indescritível, pois contém a maior biodiversidade do mundo (cerca de 100 mil espécies de invertebrados, 8200 espécies vertebrados (713 mamíferos, 1826 aves, 721 répteis, 875 anfíbios, 2800 peixes continentais e 1300 peixes marinhos) das quais 627 já estão na lista negra das espécies ameaçadas de extinção (394 espécies terrestres e 233 espécies aquáticas).",Q0,gpt-4-1106-preview,1
1,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-61-07564_63,990,1,2899447,"A fauna da Floresta Amazônica tem a maior riqueza e diversidade do Brasil.[71] Dentre os macacos são destacados os saís, os macacos-de-cheiro, os barrigudos, os guaribas e os saguis. São também bastante numerosos os cachorros-do-mato, quatis, furões, jaguatiricas e preguiças. O peixe-boi, o boto-branco e o tucuxi são animais de exclusividade dessa região. O jacamim, o pavãozinho-do-pará, a arara-vermelha, o pavão-do-mato são aves típicas da Amazônia, enquanto que dentre os ofídios é importante mencionar, por serem grandes, as jiboias e as sucuris. Dentre os peixes da Amazônia são salientados o pirarucu, as raias, tucunaré, poraquê e o baiacu. São encontrados também jacarés, tartarugas e o surubim (este em abundância ao sul da Amazônia).[71] A fauna pantaneira também se destaca quanto à diversidade de espécies animais, sendo o bioma considerado como um dos maiores centros de reprodução das Américas. Aves como o jaburu, a garça, a arara-azul-grande e o biguá; mamíferos como a onça-pintada, o cervo-do-pantanal e o lobo-guará; répteis como a sucuri; e peixes como a piramboia, o dourado e o lambari representam algumas das espécies típicas do ambiente pantaneiro.[143][144] O muriqui-do-norte é um primata endêmico do Brasil e considerado criticamente em perigo",Q0,gpt-4-1106-preview,2
2,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-21-03309_40,980,1,1021792,"São surpreendentemente uniformes, pois suas características mais óbvias são as penas e o bico córneo. São chamadas de endotérmicas , porque produzem seu próprio calor e de homoeotérmicas , porque podem manter a temperatura de seus corpos razoavelmente alta e constante. Isto não significa que a temperatura do corpo de uma ave não varie, pode haver oscilação diária de vários graus. São tetrápodas, com o par anterior transformado em asas e o posterior adaptado para empoleirar, andar ou nadar. A capacidade de voar permite às aves ocupar alguns hábitats impossíveis para outros animais. Além da quantidade, a avifauna do Brasil reúne inúmeros superlativos quanto à qualidade. Vive aqui, uma das maiores aves do mundo, a ema, ao lado das aves de menor porte, os beija-flores. Encontram-se os voadores de maior porte da Terra: o albatroz e o condor, ambos de ocorrência apenas ocasional. O gavião-real, residente no Brasil, é a ave de rapina mais possante do mundo. Ocorrem aqui as aves de vôo mais veloz: falcões e andorinhões.",Q0,gpt-4-1106-preview,3
3,Qual a maior característica da fauna brasileira?,clueweb22-pt0000-77-06884_1,970,1,3668516,"O tamanho do

In [41]:
splade_mt5_run_df[['query_id', 'Q0', "passage_id", "index", "score", "comment"]].to_csv("tests/splade_mt5_docs_000_009_50_sampled_queries_run.txt", 
                                                                                        header=None, index=False, sep=" ")

#### Compute SPLADE + mT5 NDCG@10

In [42]:
result = !/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -mndcg_cut.10 \
    {qrels_filename} "tests/splade_mt5_docs_000_009_50_sampled_queries_run.txt"

In [43]:
retrievers_results.append({'retriever':'SPLADE + mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [44]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.4546
1,BM25+E5,0.6486
2,BM25+mT5,0.7249
3,SPLADE_pt-BR,0.5900
4,ColBERT-X_mMARCO_pt-BR,0.6394
5,E5,0.5672
6,E5 base,0.4063
7,BM25 1M,0.3234
8,BM25 1M massiveweb,0.3203
9,SPLADE + mT5,0.7107
